# Project 2: Wrangle OpenStreetMap Data 

## Introduction

In this project we are obtaining, cleaning and converting map data which has been entered and edited by numerous users on OpenStreetMap. The area that I chose is Wall Township because this was my hometown. 

## Challenges encountered within the data

<font size="4">
<ul>
<li>Street names are labeled inconsistently Street for example was entered sometimes as St, st, St. or St giving us several variations 
<li>One of the zip codes was only 4 characters and so was missing a digit
<li>Many state routes run through NJ and the same state route was named in various ways such as NJ-35, Hwy 35, Rt. 35, etc. Route 9 in particular had eleven name variants.
<li>Some k tags had an added prefix with a colon giving inconsistent keys for our future table
</ul>
</font>

## Addressing the inconsistencies 

### Normalizing Street Names

<font size="4">
First I had to check what types of street names I had in order to see what needed to be normalized. To do this I used a regular expression: <code>street_type_re = re.compile(r'\b\S+\.?$')</code> to search for street types such as Avenue, Road etc. by pulling the last word of a string with or without a . to account for things like St. instead of Street. 

Next I used this function to use the regular expression to find the street type, compare it to a dictionary of expected street names, and add any unexpected street names to a variable<br>
<code> def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)</code>
            
I used this to update my dictionary, adding things like 9 for Route 9 that I missed in my initial assessment until only these truly unexpected or inconsistent street names remained:<br>
<pre>{'Ave': {'Sunset Ave', '210 5th Ave', 'Lewin Ave'},
 'Ave.': {'River Ave.'},
 'Blvd': {'Hance Blvd'},
 'C3': {'NJ-34 C3'},
 'Goldcrest': {'Goldcrest'},
 'NJ-35': {'NJ-35'},
 'NJ-88': {'NJ-88'},
 'Northbound': {'Route 9 Northbound'},
 'RT-35': {'RT-35'},
 'Rd': {'Beaver Dam Rd',
        'Chambersbridge Rd',
        'County Line Rd',
        'Herman Rd',
        'Redwood Rd',
        'Wemrock Rd'},
 'St': {'Court St', 'Bond St', 'Main St'},
 'Unit': {'North County Line Road Unit'},
 'avenue': {'elberon avenue'},
 'st': {'8th st'}</pre>
 
Next I made a function to update these street names which substitutes the inconsistent street names with the correct name that they map to.<br>
<code>def update_street_name(name, mapping=street_mapping):
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            name = re.sub(street_type_re, street_mapping[street_type], name).title()
    return name</code>
</font>

### Normalizing Zip Codes

<font size="4">
In order to find any outlying zip codes I used a regular expression as well <code>zip_code_re = re.compile(r'^\d{5}')</code> Which just finds if we have 5 digits beginning with a number which makes sense for a zip code.
Using this, I made this function which finds any zip codes that don't match our search and adds them to a variable
<code>def audit_zip_code(zips, zip_code):
    m = zip_code_re.search(zip_code)
    if not m:
        zips[zip_code].add(zip_code)</code>
        
This returned:
<pre>{'8730': {'8730'}}</pre>

Which as it turns out, is the post code listed in this data for Brielle Elementary School. A quick Google search shows that the true zip code is 08730 so it looks like they just missed a digit. To update this I made a similar function to the one above mapping the incorrect zip code to the correct one:<br>
<code>
    def update_zip_code(number, mapping=zip_mapping):
    if number in mapping:
            number = mapping.get(number)
    return number</code>
</font>

### Normalizing Route Names

<font size="4">
For the inconsistent route names found I made a similar function which gets the preferred route name from the dictionary:<br>
<code>def update_route_name(name, mapping=route_mapping):
        if name in mapping:
            name = mapping.get(name)
            return name</code>
</font>

## Analyzing the data in an SQL Database

<font size="4">
Here I aim to deterime:
<ul>
<li>The size of the database based on file length
<li>The number of unique contributors
<li>The names and contribution count of the top 10 contributors
<li>What sports facilities are in Wall and which are the most common
<li>Amenities in Wall by count
<li>What leisure items are available in Wall and their frequency
<li>The tourist attractions that are in Wall and their frequency
<li>What religions the churches practice locally
<li>The denominations of the religions and their count
<li>The most common cuisines in the area
<li>The most common cafes in the area
<li>What fast food is in the area
</ul>
</font>

In [1]:
import sqlite3
import pandas as pd

In [2]:
#Create db file and connection to sqlite3
wall_twp_db = r'C:\Users\chels\Downloads\wall_twp_db.db'
conn = sqlite3.connect(wall_twp_db)
c = conn.cursor()

### First we will find the size of our database based on file length

In [3]:
#Find our file sizes using count, make a column File Name, and output to a dataframe using pandas read to sql
print("File Size Analysis\n")
file_count = pd.read_sql('''
SELECT 'Node' as "File Name", COUNT(*) as Size FROM node
UNION ALL
SELECT 'Node Tags' as "File Name", COUNT(*)  FROM node_tags 
UNION ALL
SELECT 'Ways' as "File Name", COUNT(*) FROM ways 
UNION ALL
SELECT 'Way Nodes' as "File Name", COUNT(*) FROM way_nodes 
UNION ALL
SELECT 'Ways Tags' as "File Name", COUNT(*) FROM ways_tags
''', conn)

file_count

File Size Analysis



,File Name,Size
0,Node,1526319
1,Node Tags,120963
2,Ways,138429
3,Way Nodes,1829610
4,Ways Tags,539610


<font size="4">
The total size of all of our files is: 4,154,931

---

### Next we will find the number of unique users

In [4]:
#Count distinct users from a union of user columns in ways and nodes
user_count = c.execute('''
SELECT COUNT(DISTINCT user) 
FROM(
SELECT user FROM node
UNION 
SELECT user FROM ways
)''').fetchall()

<font size="4">
There are 1020 unique contributors.

---

### Now we will list our top 10 contributors, the top contributors appear to be bots

In [5]:
# Unify user columns from ways and node and get total contributions
top_users = pd.read_sql('''
SELECT user as User, COUNT(*) as "Total Contributions"
FROM
( 
SELECT user
FROM node
UNION ALL
SELECT user
FROM ways
) 
group by user
ORDER BY "Total Contributions" Desc
''', conn)

print("The top 10 contributors to our data are:")
top_users.head(10)

The top 10 contributors to our data are:


,User,Total Contributions
0,woodpeck_fixbot,275166
1,ppjj,246036
2,NJDataUploads,173589
3,Aurimas Fišeras,64254
4,MilkManHere,44736
5,Valustaides,44235
6,choess,35961
7,JriSv250,29478
8,TIGERcnl,22908
9,crystalwalrein,22845


---

### Looking at the local Sports Facilities it appears tennis courts are very popular

In [8]:
#find a list of sports in descending order of occurance
top_sports = pd.read_sql('''
SELECT value as "Sports Facilities", COUNT(*) as Total
FROM
( 
SELECT value
FROM node_tags
WHERE node_tags.key = "sport"
UNION ALL
SELECT value
FROM ways_tags
WHERE ways_tags.key = "sport"
) 
GROUP BY value
ORDER BY Total DESC
''', conn)

print("Here are the local sports facilities")
top_sports.head(10)

Here are the local sports facilities


,Sports Facilities,Total
0,tennis,312
1,golf,267
2,baseball,228
3,soccer,153
4,basketball,150
5,swimming,36
6,equestrian,33
7,american_football,21
8,boules,18
9,badminton,18


### Next we will look at the most common amenities

In [14]:
#find a list of amenities in descending order of occurance
top_amenity = pd.read_sql('''
SELECT value as Amenity, COUNT(*) as Total
FROM
( 
SELECT value
FROM ways_tags
WHERE ways_tags.key = "amenity"
UNION ALL
SELECT value
FROM node_tags
WHERE node_tags.key = "amenity"
) 
group by value
ORDER BY Total Desc
''', conn)

print("Here are all of our amenities in descending order")
top_amenity.head(11)

Here are all of our amenities in descending order


,Amenity,Total
0,parking,1434
1,school,576
2,parking_space,516
3,place_of_worship,510
4,restaurant,459
5,fire_station,219
6,bicycle_parking,204
7,bench,153
8,fast_food,150
9,bank,99


### Here is a list of local leisure activities. There are boardwalks with mini golf and arcades, and local theme parks

In [9]:
#find a list of leisure locations in descending order of occurance
leisure_activities = pd.read_sql('''
SELECT value as Destination, COUNT(*) as Total
FROM
( 
SELECT value
FROM ways_tags
WHERE ways_tags.key = "leisure"
UNION ALL
SELECT value
FROM node_tags
WHERE node_tags.key = "leisure"
) 
group by value
ORDER BY Total Desc
''', conn)

print("Here is a list of all of the leisure items in our data. Quite a bit to do! We are right on the coast, so the marinas and beaches are lovely!")
leisure_activities

Here is a list of all of the leisure items in our data. Quite a bit to do! We are right on the coast, so the marinas and beaches are lovely!


,Destination,Total
0,pitch,1239
1,swimming_pool,570
2,park,411
3,playground,213
4,picnic_table,156
5,golf_course,63
6,sports_centre,27
7,fitness_centre,27
8,miniature_golf,21
9,amusement_arcade,21


### For tourism there are many museums and a couple of theme parks that catch my eye.

In [15]:
#find a list of tourist items in descending order of occurance
tourism = pd.read_sql('''
SELECT value as Destination, COUNT(*) as Total
FROM
( 
SELECT value
FROM ways_tags
WHERE ways_tags.key = "tourism"
UNION ALL
SELECT value
FROM node_tags
WHERE node_tags.key = "tourism"
) 
group by value
ORDER BY Total Desc
''', conn)

print("Tourist Attractions")
tourism

Tourist Attractions


,Destination,Total
0,hotel,51
1,attraction,42
2,museum,36
3,motel,21
4,artwork,18
5,picnic_site,15
6,gallery,15
7,guest_house,12
8,viewpoint,6
9,camp_site,6


### Here are the religions practiced by the local churches and the count of the churches:

In [12]:
#find a list of church faiths and their frequency
churches = pd.read_sql('''
SELECT value as Religion, COUNT(*) as Total
FROM
( 
SELECT value
FROM ways_tags
WHERE ways_tags.key = "religion"
AND ways_tags.id IN 
(
SELECT id
FROM ways_tags
WHERE ways_tags.value = "place_of_worship"
)
UNION ALL
SELECT value
FROM node_tags
WHERE node_tags.key = "religion"
AND node_tags.id IN 
(
SELECT id
FROM node_tags
WHERE node_tags.value = "place_of_worship"
)
) 
group by value
ORDER BY Total Desc
''', conn)

print("Churches")
churches

Churches


,Religion,Total
0,christian,405
1,jewish,39
2,muslim,3


### Here is a list of all of the local religious denominations. I grew up methodist and now I know that must be quite common!

In [13]:
# Make a list of religious denominations per church and their frequency
denominations = pd.read_sql('''
SELECT value as Denomination, COUNT(*) as Total
FROM
( 
SELECT value
FROM ways_tags
WHERE ways_tags.key = "denomination"
AND ways_tags.id IN 
(
SELECT id
FROM ways_tags
WHERE ways_tags.value = "place_of_worship"
)
UNION ALL
SELECT value
FROM node_tags
WHERE node_tags.key = "denomination"
AND node_tags.id IN 
(
SELECT id
FROM node_tags
WHERE node_tags.value = "place_of_worship"
)
) 
group by value
ORDER BY Total Desc
''', conn)

print("Religious Denominations")
denominations

Religious Denominations


,Denomination,Total
0,methodist,84
1,baptist,57
2,orthodox,24
3,catholic,24
4,lutheran,15
5,roman_catholic,12
6,pentecostal,9
7,presbyterian,6
8,jehovahs_witness,6
9,nondenominational,3


### No surprises here. This list shows pizza and italian as being among the highest in count of cuisines. It is a very italian area.

In [16]:
# List of cuisines by occurance
cuisines = pd.read_sql('''
SELECT value as Cuisine, COUNT(*) as Total
FROM
( 
SELECT value
FROM ways_tags
WHERE ways_tags.key = "cuisine"
AND ways_tags.id IN 
(
SELECT id
FROM ways_tags
WHERE ways_tags.value = "restaurant"
OR ways_tags.value = "fast_food"
)
UNION ALL
SELECT value
FROM node_tags
WHERE node_tags.key = "cuisine"
AND node_tags.id IN 
(
SELECT id
FROM node_tags
WHERE node_tags.value = "restaurant"
OR node_tags.value = "fast_food"
)
) 
group by value
ORDER BY Total Desc
''', conn)

print("Cusines")
cuisines

Cusines


,Cuisine,Total
0,pizza,45
1,italian,33
2,burger,33
3,mexican,18
4,sandwich,15
5,seafood,12
6,american,12
7,donut;coffee_shop,9
8,donut,9
9,thai,6


### Here is list of local cafes by occurance.

#### Initially this surprised me very much, seeing as how I thought Dunkin' Donuts would be king in my hometown.
#### This is actually due to a data inconsistancy where D&D is sometimes listed as just fast food and not a cafe, as we will see in the next section

In [18]:
# List of cafes by occurance
cafes = pd.read_sql('''
SELECT value as Cafe, COUNT(*) as Total
FROM
( 
SELECT value
FROM ways_tags
WHERE ways_tags.key = "name"
AND ways_tags.id IN 
(
SELECT id
FROM ways_tags
WHERE ways_tags.value = "cafe"
)
UNION ALL
SELECT value
FROM node_tags
WHERE node_tags.key = "name"
AND node_tags.id IN 
(
SELECT id
FROM node_tags
WHERE node_tags.value = "cafe"
)
) 
group by value
ORDER BY Total Desc
''', conn)

print("Cafes")
cafes

Cafes


,Cafe,Total
0,Starbucks,15
1,Playa Bowls,9
2,Dunkin' Donuts,6
3,Playa Bowls Ocean,3
4,Ms. Bagel,3
5,Inkwell Coffeehouse,3
6,Green Planet Coffee,3
7,Catsbury Park,3
8,Cathy's Bagels,3
9,Cafe Volan,3


### Here is a list of the fast food locations in the area. Now this makes more sense!
#### There are a ton more Dunkin' Donuts in the area than Starbucks, they are just listed as fast food! I guess they can be both!
#### Living in Seattle now, I don't know where my loyalties lie!


In [ ]:
# List of fast food by occurance
fast_food = pd.read_sql('''
SELECT value as "Fast Food Restaurant", COUNT(*) as Total
FROM
( 
SELECT value
FROM ways_tags
WHERE ways_tags.key = "name"
AND ways_tags.id IN 
(
SELECT id
FROM ways_tags
WHERE ways_tags.value = "fast_food"
)
UNION ALL
SELECT value
FROM node_tags
WHERE node_tags.key = "name"
AND node_tags.id IN 
(
SELECT id
FROM node_tags
WHERE node_tags.value = "fast_food"
)
) 
group by value
ORDER BY Total Desc
''', conn)

print("Now this makes more sense! Here is a list of the fast food locations in the area.\nThere are a ton more Dunkin' Donuts in the area than Starbucks, they are just listed as fast food! I guess they can be both!\nLiving in Seattle now, I don't know where my loyalties lie!")
fast_food

---
## Further improvments we could make to our dataset
---

### Normalzing name_type and name_base in ways_tags file

#### Initially I went to search how many of each street type we had and found that there was street names seperated by colons or semicolons

In [53]:
pd.read_sql('''
SELECT value, COUNT(*) as Total
FROM ways_tags
WHERE ways_tags.key = "name_type"
AND value LIKE "%;%"
UNION ALL
SELECT value, COUNT(*) as Total
FROM ways_tags
WHERE ways_tags.key = "name_type"
AND value LIKE "%:%"
GROUP BY value
ORDER BY Total DESC
''', conn)

,value,Total
0,Pl;Ave,12
1,Ave:St,6
2,Pky:St,3
3,Ave:Way,3


#### I thought these could just be updated with a dictionary as we have already done, but after checking on Heritage Boulevard I found it has a name_type of Ave:Way when it is a Boulevard and a name_base of Heritage:Volunteer where we can't be sure where the Volunteer is coming from. A search shows that Heritage Boulevard exists in this zip code and a Heritage Ave or Volunteer Way does not.

In [44]:
pd.read_sql('''
SELECT key, value
FROM ways_tags
WHERE id = 11699202
and key = "name"
UNION ALL
SELECT key, value
FROM ways_tags
WHERE id = 11699202
and key = "name_type"
UNION ALL
SELECT key, value
FROM ways_tags
WHERE id = 11699202
and key = "name_base"
''', conn)

,key,value
0,name,Heritage Boulevard
1,name,Heritage Boulevard
2,name,Heritage Boulevard
3,name_type,Ave:Way
4,name_type,Ave:Way
5,name_type,Ave:Way
6,name_base,Heritage:Volunteer
7,name_base,Heritage:Volunteer
8,name_base,Heritage:Volunteer



#### This being as it is, we can't just update a dictionary to update all Ave:Way to Boulvards just in case the other Ave:Way's are something else. We must make a program that instead finds the street types using the regular expression we already made and sets those to the name_base and make a new regular expression that finds everything before that and then make a function to add that to the name_type just in case so we can be sure those values are correct.. If we normalized the types and bases we could figure out how many of each street type we had, and what the most common street types are as well as other things, like how long is the average street name and much more.
---

### Normalizing cuisine categories

#### The cuisines are frequently seperated by semi-colons


In [81]:
# List of cuisines by occurance
pd.read_sql('''
SELECT value as Cuisine, COUNT(*) as Total
FROM
( 
SELECT value
FROM ways_tags
WHERE ways_tags.key = "cuisine"
AND value LIKE "%;%"
AND ways_tags.id IN 
(
SELECT id
FROM ways_tags
WHERE ways_tags.value = "restaurant"
OR ways_tags.value = "fast_food"
)
UNION ALL
SELECT value
FROM node_tags
WHERE node_tags.key = "cuisine"
AND value LIKE "%;%"
AND node_tags.id IN 
(
SELECT id
FROM node_tags
WHERE node_tags.value = "restaurant"
OR node_tags.value = "fast_food"
)
) 
group by value
ORDER BY Total Desc
''', conn)

print("Cusines")
cuisines

Cusines


,Cuisine,Total
0,donut;coffee_shop,9
1,donut;coffee,6
2,regional;Fried_foods;desserts,3
3,pizza;italian,3
4,pizza;dessert;american;dairy;coffee_shop,3
5,mexican;californian,3
6,japanese;sushi;steak,3
7,italian;pizza,3
8,italian;burger,3
9,fancy;dairy,3


#### As we can see above, it would be difficult to sort by a breakfast place, when some places have breakfast folded into one large string of other categories. In order to fix this and make the data usable we would have to create a new table with our cuisine types and restaraunts and programatically seperate our cuisines by semicolon and add them to the table by restaraunt name.
---